In [22]:
import time
from datetime import datetime

from nn import MLP
from micrograd import Value

In [23]:
n = MLP(784, [128, 10])

## Step 1: Load and Prepare the Data

MNIST images are 28×28 pixels = 784 values. We need to:
1. Load the data
2. **Flatten** each image: (28, 28) → 784-element vector
3. **Normalize** pixels: 0-255 → 0.0-1.0 (helps training)
4. Convert to Value objects (for autodiff)

In [24]:
from mnist_loader import load_mnist

# Load the data
X_train, y_train, X_test, y_test = load_mnist()

print(f"Training data: {X_train.shape}")  # (60000, 28, 28)
print(f"Training labels: {y_train.shape}")  # (60000,)
print(f"Pixel range: {X_train.min()} to {X_train.max()}")  # 0 to 255

Training data: (60000, 28, 28)
Training labels: (60000,)
Pixel range: 0 to 255


In [25]:
# Start small - use only 100 samples for fast experimentation
n_samples = 10
X_small = X_train[:n_samples]
y_small = y_train[:n_samples]

# Flatten: (100, 28, 28) → (100, 784)
# reshape(-1, 784) means: keep batch size automatic, make each row 784 elements
X_flat = X_small.reshape(-1, 784)

# Normalize: 0-255 → 0.0-1.0
# Why? Keeps gradients stable. Large numbers (255) can cause exploding gradients
X_norm = X_flat / 255.0

print(f"Flattened shape: {X_flat.shape}")  # (100, 784)
print(f"Normalized range: {X_norm.min():.2f} to {X_norm.max():.2f}")  # 0.00 to 1.00

Flattened shape: (10, 784)
Normalized range: 0.00 to 1.00


## Step 2: Define the Loss Function

**What's a loss?** A number that measures how wrong your predictions are.
- Low loss = good predictions
- High loss = bad predictions

### Loss Function Comparison

Imagine you're classifying a digit as "3".

Your network outputs 10 scores (one for each digit 0-9).

Let's say the raw scores are:
`[1.2, 0.5, 2.1, 4.5, 1.8, 0.9, 1.1, 2.0, 0.7, 1.3]`

The correct answer is digit 3 (index 3, which has score 4.5 - the highest!). Now let's see how each loss function "thinks" about this:

#### 1) **MSE Loss (Mean Squared Error)**
MSE wants your outputs to match a target vector exactly.

First, convert the label to **one-hot encoding**:
- Label: 3 → Target: `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`
- (Only the correct class gets 1, all others get 0)

Your output: `[1.2, 0.5, 2.1, 4.5, 1.8, 0.9, 1.1, 2.0, 0.7, 1.3]`

MSE calculates: `(1.2-0)² + (0.5-0)² + (2.1-0)² + (4.5-1)² + ...`

The problem:
- It's angry that digit 2 has score 2.1 instead of 0.0
- It's angry that digit 3 has score 4.5 instead of 1.0
- It treats these errors equally!

**What MSE is saying**: "I don't care that you picked the right digit. Your numbers are wrong!"

It's like a teacher marking your test wrong because you wrote "3.0" instead of exactly "3".

#### 2) **Hinge Loss (Multi-class SVM)**
Hinge loss asks: "Is the correct class winning by at least a margin of 1.0 against EVERY wrong class?"

**Formula**: `L = Σ(i≠y) max(0, s_i - s_y + 1)` 
- where `y` = correct class index (3)
- `s_y` = score of correct class (4.5)
- `s_i` = score of each wrong class
- We sum up penalties for ALL wrong classes that are too close

**Complete Dry Run:**

Scores: `[1.2, 0.5, 2.1, 4.5, 1.8, 0.9, 1.1, 2.0, 0.7, 1.3]`  
True label: 3 (so s_y = 4.5)

For each wrong class i, calculate: max(0, s_i - 4.5 + 1)

- Class 0: max(0, 1.2 - 4.5 + 1) = max(0, -2.3) = **0** ✓
- Class 1: max(0, 0.5 - 4.5 + 1) = max(0, -3.0) = **0** ✓
- Class 2: max(0, 2.1 - 4.5 + 1) = max(0, -1.4) = **0** ✓
- ~~Class 3~~: (skip - this is the correct class)
- Class 4: max(0, 1.8 - 4.5 + 1) = max(0, -1.7) = **0** ✓
- Class 5: max(0, 0.9 - 4.5 + 1) = max(0, -2.6) = **0** ✓
- Class 6: max(0, 1.1 - 4.5 + 1) = max(0, -2.4) = **0** ✓
- Class 7: max(0, 2.0 - 4.5 + 1) = max(0, -1.5) = **0** ✓
- Class 8: max(0, 0.7 - 4.5 + 1) = max(0, -2.8) = **0** ✓
- Class 9: max(0, 1.3 - 4.5 + 1) = max(0, -2.2) = **0** ✓

**Total Loss = 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 = 0** ✓

**What Hinge Loss is saying**: "You got it right by a comfortable margin (at least 1.0 from every wrong class). Perfect!"

**The limitation - A Different Example:**

Now imagine the scores were: `[2.0, 1.0, 2.0, 4.1, 2.0, 1.5, 1.0, 3.9, 1.0, 2.0]`  
True label: still 3 (so s_y = 4.1)

Notice: Class 7 has score 3.9, very close to the correct answer!

Let's recalculate:

- Class 0: max(0, 2.0 - 4.1 + 1) = max(0, -1.1) = **0** ✓
- Class 1: max(0, 1.0 - 4.1 + 1) = max(0, -2.1) = **0** ✓
- Class 2: max(0, 2.0 - 4.1 + 1) = max(0, -1.1) = **0** ✓
- ~~Class 3~~: (skip - correct class)
- Class 4: max(0, 2.0 - 4.1 + 1) = max(0, -1.1) = **0** ✓
- Class 5: max(0, 1.5 - 4.1 + 1) = max(0, -1.6) = **0** ✓
- Class 6: max(0, 1.0 - 4.1 + 1) = max(0, -2.1) = **0** ✓
- Class 7: max(0, 3.9 - 4.1 + 1) = max(0, **0.8**) = **0.8** ⚠️ PENALTY!
- Class 8: max(0, 1.0 - 4.1 + 1) = max(0, -2.1) = **0** ✓
- Class 9: max(0, 2.0 - 4.1 + 1) = max(0, -1.1) = **0** ✓

**Total Loss = 0 + 0 + 0 + 0 + 0 + 0 + 0.8 + 0 + 0 = 0.8** ⚠️

**The problem**: Even though you predicted correctly (class 3 is still the highest), hinge loss penalizes you because class 7 is too close. The margin between them is only 0.2 (4.1 - 3.9), but hinge loss wants at least 1.0.

This forces the network to be "confidently correct" - not just barely winning.

#### 3) **Softmax + Cross-Entropy**
This is a two-step process:

**Step 1: Softmax** converts scores to probabilities:
```
Raw scores: [1.2, 0.5, 2.1, 4.5, 1.8, 0.9, 1.1, 2.0, 0.7, 1.3]
Probabilities: [0.04, 0.02, 0.09, 0.70, 0.06, 0.03, 0.03, 0.08, 0.02, 0.04]
```
(They now sum to 1.0 - these are "confidences")

**Step 2: Cross-Entropy** asks: "What probability did you assign to the correct class?"

- You said: 0.70 (70% confident it's a 3)
- Loss = -log(0.70) = 0.36

If you had been more confident:
- 90% confident → Loss = -log(0.90) = 0.11 (better!)
- 99% confident → Loss = -log(0.99) = 0.01 (even better!)

If you had been less confident:
- 50% confident → Loss = -log(0.50) = 0.69 (worse!)
- 10% confident → Loss = -log(0.10) = 2.30 (much worse!)
- 1% confident → Loss = -log(0.01) = 4.61 (terrible!)

**What Cross-Entropy is saying**: "You got it right, but you're only 70% sure. Let's push you to be MORE confident in the correct answer."

It's never satisfied! Even at 99.9% it's saying "you can do better."

---

### Summary Comparison

| Loss Function | What it optimizes | When to use | Pros | Cons |
|--------------|-------------------|-------------|------|------|
| **MSE** | Match exact target values | Regression, simple cases | Simple, easy to implement | Not ideal for classification |
| **Hinge Loss** | Win by a margin | Binary/multi-class SVM | Forces confident boundaries | Doesn't care beyond margin |
| **Cross-Entropy** | Maximize probability of correct class | Multi-class classification | Industry standard, best performance | Requires softmax, slightly complex |

**For this tutorial**: We'll start with **MSE** (simplest to understand and implement). Once you see it working, you can experiment with the others!

In [28]:
def loss_fn() -> tuple[Value, float]:
    inputs = [list(map(Value, xrow)) for xrow in X_norm]

    predictions = []

    for i, x in enumerate(inputs):
        t0 = time.time()
        predictions.append(n(x))
        t1 = time.time()
        print(f"[{i} - {datetime.utcnow().isoformat()}] Time taken: {t1 - t0:.3f}s")

    # For each sample:
    #   - Get the true label (y_small[i])
    #   - Create one-hot target: [0,0,0,1,0,0,0,0,0,0] if label is 3
    #   - Calculate (pred - target)^2 for all 10 outputs
    #   - Sum them up
    # Then average across all samples

    losses = []
    for i in range(n_samples):
        # Get prediction (list of 10 Values)
        pred = predictions[i]

        # Get true label
        true_label = y_small[i]

        target = [0] * 10
        target[true_label] = 1

        sample_loss = sum((pred[j] - target[j]) ** 2 for j in range(10))
        losses.append(sample_loss)

    total_loss = sum(losses) / (1.0 * n_samples)

    # For each sample, check if argmax(prediction) == true_label
    # Hint: Find which output has the highest .data value
    correct = 0.0
    for i in range(n_samples):
        predicted_digit = max(range(10), key=lambda j: predictions[i][j].data)

        if predicted_digit == y_small[i]:
            correct += 1

    return total_loss, correct / n_samples


loss_fn()

/var/folders/f3/9g0xpf9j7_3ctlwn4fwzcwjc0000gn/T/ipykernel_79904/838481714.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{i} - {datetime.utcnow().isoformat()}] Time taken: {t1 - t0:.3f}s")


[0 - 2025-11-23T08:32:03.781134] Time taken: 0.176s
[1 - 2025-11-23T08:32:03.947789] Time taken: 0.167s
[2 - 2025-11-23T08:32:04.116396] Time taken: 0.169s
[3 - 2025-11-23T08:32:04.295041] Time taken: 0.179s
[4 - 2025-11-23T08:32:04.458621] Time taken: 0.164s
[5 - 2025-11-23T08:32:04.634666] Time taken: 0.176s
[6 - 2025-11-23T08:32:04.799985] Time taken: 0.165s
[7 - 2025-11-23T08:32:04.979981] Time taken: 0.180s
[8 - 2025-11-23T08:32:05.153571] Time taken: 0.174s
[9 - 2025-11-23T08:32:05.337028] Time taken: 0.183s


(Value(data=9.773719152846573), 0.0)

## Step 4: Training Loop (Gradient Descent)

Now we train the network! This is where the magic happens.

**The Training Algorithm:**
1. **Forward pass**: Calculate loss (measure how wrong we are)
2. **Backward pass**: Calculate gradients (which direction to adjust weights)
3. **Update weights**: Move in the opposite direction of gradients

**Gradient Descent Intuition:**
- Imagine you're lost in foggy mountains trying to get to the bottom (minimum loss)
- You can't see far, but you can feel which way is downhill (gradient)
- Take a small step downhill (learning_rate × gradient)
- Repeat until you reach the bottom!

**The gradient** tells us: "If you increase this weight, loss will increase/decrease by this much"

We want to **decrease** loss, so we move weights in the **opposite** direction of the gradient.

In [ ]:
# Training loop
learning_rate = 0.01  # How big of a step to take
n_steps = 5  # Number of training iterations

for k in range(n_steps):
    total_loss, accuracy = loss_fn()

    n.zero_grad()

    total_loss.backward()

    for p in n.parameters():
        p.data -= learning_rate * p.grad

    # Print progress every 10 steps
    if k % 1 == 0:
        print(f"step {k}: loss {total_loss.data:.4f}, accuracy {accuracy:.2%}")

/var/folders/f3/9g0xpf9j7_3ctlwn4fwzcwjc0000gn/T/ipykernel_79904/838481714.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{i} - {datetime.utcnow().isoformat()}] Time taken: {t1 - t0:.3f}s")


[0 - 2025-11-23T08:33:23.262171] Time taken: 0.186s
[1 - 2025-11-23T08:33:23.427211] Time taken: 0.165s
[2 - 2025-11-23T08:33:23.614672] Time taken: 0.187s
[3 - 2025-11-23T08:33:23.828028] Time taken: 0.213s


## Step 5: Test the Trained Model

After training, let's see what the network learned! We'll make predictions on a few examples.

In [ ]:
# Test on first 5 samples
print("Testing trained model on first 5 samples:\n")

for i in range(min(5, n_samples)):
    # Get input and convert to Values
    x_test = list(map(Value, X_norm[i]))

    # Forward pass
    pred = n(x_test)

    # Find predicted digit (argmax)
    predicted_digit = max(range(10), key=lambda j: pred[j].data)
    true_digit = y_small[i]

    # Show prediction vs truth
    status = "✓" if predicted_digit == true_digit else "✗"
    print(f"Sample {i}: True={true_digit}, Predicted={predicted_digit} {status}")

    # Show the scores for all 10 digits
    print(f"  Scores: {[f'{pred[j].data:.2f}' for j in range(10)]}")
    print()